# Human genomics pipeline

An example of genetic analyis and BLAST on human genome.

As usual, we need the sequence and the annotations (both name and biotype).

In [89]:
library(Biostrings)
library(BSgenome)
library(biomaRt)
library(dplyr)


ERROR: Error in library(tidyverse): there is no package called ‘tidyverse’


Annotation in humans:

https://www.google.com/search?q=bioconductor+annotation+packages

https://bioconductor.org/packages/release/data/annotation/

Masked genomes helps, because in them repeats are *masked*, so less likely to scramble alignments. For the sequences, we need the sequences (that can be downloaded as FASTA file) of the genome, and the annotation (.gff file), that can be downloaded separately.

Another good source of genomes is UniCal Santa Cruz Genomic Institute website. Not a lot of species, tho.

Soft masking: the repeated regions are lowercase in the FASTA file, and many programs can recognize soft masking. Most used in comparative genomic, when doing whole genome comparisons.

# Loading data and required packages

Loading human genome, subsetting sequences on chromosome, skipping small sequences and unknowns.

In [2]:
#BiocManager::install("BSgenome.Hsapiens.UCSC.hg38")
library(BSgenome.Hsapiens.UCSC.hg38)
# after installation of the whole genome (700 MB)...
genome <- BSgenome.Hsapiens.UCSC.hg38
chr    <- seqnames(genome)  #get the sequence names
chr    <- chr[-grep("_", chr)] # genome chromosomic indexes : chr1, chr2... chrX, chrY, chrM


Loading annotations. Alternatively, you can download annotation in GFF3 format from Ensembl FTP repository. But the best option will be to get everything from BiomaRt.

In [30]:
library(biomaRt)

#set our interval analysis
upstream   <- 500
downstream <- 100

#get protein coding sequences and annotations
ensembl <- useMart("ENSEMBL_MART_ENSEMBL")
ensembl <- useDataset("hsapiens_gene_ensembl", mart = ensembl)  #found with : 
#    searchDatasets(mart =ensembl, pattern = "hsapiens")

#build the query:

get.coding.genes.info = function() {
            getBM(mart = ensembl,
                           attributes = c("ensembl_gene_id", 
                           "chromosome_name", 
                           "start_position", 
                           "end_position", 
                           "strand"),    # to know fields, searchAttributes 
            filters ="biotype",
            values="protein_coding")
}

gene.tab = get.coding.genes.info()

In [31]:
gene.tab.backup <- gene.tab

In [43]:
#array di cromosomi
# gia' ripulito chr
chr.length = list()
for (i in chr) chr.length[[i]] = length(genome[[i]])


In [44]:
head(chr.length)

$chr1
[1] 248956422

$chr2
[1] 242193529

$chr3
[1] 198295559

$chr4
[1] 190214555

$chr5
[1] 181538259

$chr6
[1] 170805979

In [32]:
head(unique(gene.tab$chromosome_name))

[1] "MT"                          "CHR_HSCHR19LRC_LRC_T_CTG3_1"
[3] "KI270734.1"                  "CHR_HSCHR15_4_CTG8"         
[5] "GL000009.2"                  "KI270728.1"

Chromosome names in gene.tab (taken from UCSC) are different from the names of the chromosome (taken from Ensembl)! 
"chr20" vs "20". We can just paste "chr", but there will be problems with mitocondrial ("chrM" vs "MT"). We should really write a function for that.

In [33]:
gene.tab$chromosome_name = paste("chr",gene.tab$chromosome_name, sep="")

Now we calculate promoters positions.

In [34]:
names(gene.tab)

[1] "ensembl_gene_id" "chromosome_name" "start_position"  "end_position"   
[5] "strand"

In [35]:
#conditional computation of start and end positions:

gene.tab$prom.start <- ifelse(gene.tab$strand=="1",
                              gene.tab$start_position - upstream  ,
                              gene.tab$end_position   - downstream) # set promoter starts

gene.tab$prom.end   <- ifelse(gene.tab$strand=="-1",
                              gene.tab$start_position + downstream,
                              gene.tab$end_position + upstream)   # set promoter ends


In [36]:
names(gene.tab)
head(gene.tab)

[1] "ensembl_gene_id" "chromosome_name" "start_position"  "end_position"   
[5] "strand"          "prom.start"      "prom.end"

ensembl_gene_id,chromosome_name,start_position,end_position,strand,prom.start,prom.end
<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>
ENSG00000198888,chrMT,3307,4262,1,2807,4762
ENSG00000198763,chrMT,4470,5511,1,3970,6011
ENSG00000198804,chrMT,5904,7445,1,5404,7945
ENSG00000198712,chrMT,7586,8269,1,7086,8769
ENSG00000228253,chrMT,8366,8572,1,7866,9072
ENSG00000198899,chrMT,8527,9207,1,8027,9707


In [154]:
# BUGGED: check if all sequences are in boundaries

to.delete = apply(gene.tab, 1, function(x){
    as.numeric(x[7]) > chr.length[[as.character(x[2])]]}, simplify = T)

ERROR: Error in FUN(newX[, i], ...): unused argument (simplify = T)


In [37]:
gene.tab <- gene.tab %>% 
    filter(grepl("^chr[1-9*]|Y|X$", chromosome_name)) %>%
    mutate(strand = ifelse(strand=="1", "+", "-")) 

unique(gene.tab$chromosome_name)
unique(gene.tab$strand)

[1] "chr5"  "chr9"  "chrY"  "chr4"  "chr13" "chr11" "chrX"  "chr14" "chr20"
[10] "chr6"  "chr15" "chr7"  "chr10" "chr19" "chr2"  "chr17" "chr3"  "chr1" 
[19] "chr12" "chr16" "chr8"  "chr21" "chr18" "chr22"

[1] "-" "+"

In [38]:
gene.tab$prom.seq <- getSeq(genome,
                            names=gene.tab$chromosome_name,
                            start= gene.tab$prom.start,
                            end=gene.tab$prom.end,
                            strand=gene.tab$strand,
                            as.character=T
                            )

ERROR: Error in .Call2("solve_user_SEW", refwidths, start, end, width, translate.negative.coord, : solving row 1: the supplied start/end lead to a negative width


In [160]:
#rm(to.delete)
gene.tab2 <- gene.tab[!to.delete]

ERROR: Error in !to.delete: invalid argument type


In [140]:
gene.tab2

Warning message in rbind(parts$upper, ellip_v, parts$lower, deparse.level = 0L):
“number of columns of result is not a multiple of vector length (arg 2)”

1    
2    
3    
4    
5    
6    
7    
8    
9    
10   
11   
12   
13   
14   
15   
16   
17   
18   
19   
20   
21   
22   
23   
24   
25   
26   
27   
28   
29   
30   
⋮    
22778
22779
22780
22781
22782
22783
22784
22785
22786
22787
22788
22789
22790
22791
22792
22793
22794
22795
22796
22797
22798
22799
22800
22801
22802
22803
22804
22805
22806
22807

In [ ]:
dplyr branch pipe

In [71]:
chrs <- as.data.frame(t(as.data.frame(chr.length)))
colnames(chrs) = "chr_length"

chrs <- chrs %>% 
    tibble::rownames_to_column() %>% 
    mutate(chromosome_name=rowname) %>%
    select(-rowname)

chrs

chr_length,chromosome_name
<int>,<chr>
248956422,chr1
242193529,chr2
198295559,chr3
190214555,chr4
181538259,chr5
170805979,chr6
159345973,chr7
145138636,chr8
138394717,chr9


In [88]:
#or, with dplyr piping...
# It does not change/overwrite the original data, unless requested.

gene.tab.backup %>%


    
    mutate(chromosome_name = paste("chr",chromosome_name, sep="")) %>% # this is an error source. verify matches.

    filter(grepl("^chr[1-9*]|Y|X$", chromosome_name)) %>%

    mutate(strand = ifelse(strand=="1", "+", "-")) %>%

    mutate(prom.start = ifelse(.$strand=="+",dplyr branch pipe
                              .$start_position - upstream  ,
                              .$end_position   - downstream)) %>%
    mutate(prom.end=    ifelse(.$strand=="-",
                              .$start_position + downstream,
                              .$end_position + upstream)) %>%

    left_join (chrs, by = "chromosome_name") %>%

    # clean out of boundaries

    filter(prom.start>0) %>% 
    filter(prom.end<chr_length) %>%

    head() 

ensembl_gene_id,chromosome_name,start_position,end_position,strand,prom.start,prom.end,chr_length
<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<int>
ENSG00000286094,chr5,716808,766919,-,766819,716908,181538259
ENSG00000204805,chr9,61861625,61863200,+,61861125,61863700,138394717
ENSG00000286265,chrY,19076946,19077416,-,19077316,19077046,57227415
ENSG00000285447,chr9,112997120,113050043,-,113049943,112997220,138394717
ENSG00000288380,chr4,1391552,1395992,+,1391052,1396492,190214555
ENSG00000139675,chr13,52642425,52643796,+,52641925,52644296,114364328


to do: debug, and plot the consensus matrix plot.
you'll see that the pattern is different. 
high body temperature organism has higher G-C content, easier to work on.
the percentage of GC and AT is linked to thermodinamical context. The consensum matrix help in predicting the start point of transcription on genome.

In [ ]:
#homework: check the snake